### Elie NOUHRA - TD noté matière "SVM et ANN" - Master 2 ECAP IAE NANTES
Python 3.9.13

## TD N°2 explicabilité du dataset Boston Housing

1) Charger le dataset bostong_housing
   1) Disponible dans le folder 04_interpretable_ml/td/data/
   
2) Nettoyer votre jeu de données pour créer une régression linéaire et un random forest
   1) Tester d'ajouter des features log, quadratique, ...

3)Créer un modèle baseline linéaire et random forest

4) Interpréter le modèle linéaire

5) Tuner votre random forest

6) Interpréter globalement votre modèle meilleur modèle RF 
   1) Utiliser les PDP ou ALE & Permutation feature Importance 
   2) Comparer les résulats du random forest avec votre interprétation du modèle linéaire

6) Réaliser une explicabilité par individu
   1) En utilisant la méthode ICE (PDP individuelle)
   2) LIME (Model local pour expliquer une prédiction)
   3) SHAP watterfall plot (Contribution marginale de chaque variable dans la prédiction)

7) Réaliser une explicabilité par individu sur le modèle RF
- 1) ICE, le PDP est-il une bonne représentation des variables importantes de votre modèle?
- 2) LIME
- 3) SHAP watterfall plot

8) Explorer les graphiques SHAP étudiés  dans la partie CM
   1) beeswarm (Contribution des variables)
   2) scatter (équivalent pdp)

## Contexte du Dataset

Le Boston Housing dataset est un ensemble de données couramment utilisé en apprentissage automatique et en statistique pour étudier les relations entre diverses caractéristiques socio-économiques et immobilières dans la ville de Boston.  
Il contient des informations sur des propriétés résidentielles et leur environnement, et est souvent utilisé pour prédire la valeur des maisons, un problème classique de régression.

**Variable dispo**: 
- CRIM : taux de criminalité par habitant par ville
- ZN : proportion de terrains résidentiels zonés pour des lots de plus de 25 000 pieds carrés
- INDUS : proportion de terrains commerciaux non commerciaux par ville
- CHAS : variable binaire indiquant la proximité de la rivière Charles (= 1 si la zone délimitée par la ville touche la rivière ; 0 sinon)
- NOX : concentration des oxydes d'azote (en parties par 10 millions)
- RM : nombre moyen de pièces par logement
- AGE : proportion des unités occupées par leur propriétaire et construites avant 1940
- DIS : distances pondérées vers cinq centres d'emploi de Boston
- RAD : indice d'accessibilité aux autoroutes radiales
- TAX : taux d'imposition foncière par valeur totale pour chaque tranche de 10 000 dollars
- PTRATIO : ratio élèves-enseignants par ville
- LSTAT : pourcentage de la population de statut socio-économique inférieur
- MEDV : valeur médiane des maisons occupées par leur propriétaire (en milliers de dollars) - **variable cible**

In [28]:
#Web request
import requests
import io
import pandas as pd

### 1) Charger le dataset bostong_housing

In [29]:
#Télécharge directement depuis Github
url = "https://raw.githubusercontent.com/Roulitoo/cours_iae/master/04_INTERPRETABLE_ML/td/data/boston_housing.csv" 
download = requests.get(url).content

data = pd.read_csv(io.StringIO(download.decode('utf-8')), sep=';')

In [ ]:
# Aperçu des données
data.info()
print(data.head())
print(data.tail())

In [ ]:
# Remove 'Unnamed' column 
data_cleaned = data.loc[:, ~data.columns.str.contains('^Unnamed')]

# Move 'MEDV' column to the first position
medv = data_cleaned.pop('MEDV')
data_cleaned.insert(0, 'MEDV', medv)

data_cleaned.info()

### 2)Nettoyer votre jeu de données pour créer une régression linéaire et un random forest

Penser à :

- Vérifier comment encoder vos variables qualitatives pour la modélisation 
- Analyser les distributions
- Analyser les outliers 
- Analyser les corrélations

>Tester d'ajouter des features log, quadratique, ...

#### VALEURS MANQUANTES

In [ ]:
import seaborn as sns

# Check for missing values
missing_values = data_cleaned.isnull().sum()
print("Missing values in each column :\n", missing_values)

#### DISTRIBUTION DES DONNÉES

In [ ]:
data_cleaned.nunique()

L'indice d'accessibilité aux autoroutes radiales (RAD) ne prends que 9 valeurs distinctes > passage en catégorielle

In [ ]:
data_cleaned['RAD'] = data_cleaned['RAD'].astype('category')

In [ ]:
# Visualisation variables catégorielles
import matplotlib.pyplot as plt

# Analyse de la variable CHAS
chas_counts = data_cleaned['CHAS'].value_counts()
chas_counts.plot(kind='bar', figsize=(8, 6))
plt.title('Bar Plot of CHAS')
plt.xlabel('CHAS')
plt.ylabel('Count')
plt.show()

# Nombre et pourcentage des valeurs dans CHAS
chas_percentage = data_cleaned['CHAS'].value_counts(normalize=True) * 100
print("CHAS Counts:\n", chas_counts)
print("CHAS Percentage:\n", chas_percentage)

# Analyse de la variable RAD
rad_counts = data_cleaned['RAD'].value_counts()
rad_counts.plot(kind='bar', figsize=(8, 6))
plt.title('Bar Plot of RAD')
plt.xlabel('RAD')
plt.ylabel('Count')
plt.show()

# Nombre et pourcentage des valeurs dans RAD
rad_percentage = data_cleaned['RAD'].value_counts(normalize=True) * 100
print("RAD Counts:\n", rad_counts)
print("RAD Percentage:\n", rad_percentage)

Seulement 7% des maisons sont à proximité de la rivière Charles. L'indice d'accessibilité aux autoroutes radiales varie et la valeur 24 décroche avec les autres valeurs qui sont plus faibles.

In [ ]:
# Create boxplots
import matplotlib.pyplot as plt

# Exclude categorical  columns
columns_to_plot = data_cleaned.columns.difference(['CHAS', 'RAD'])

# Boxplots
data_cleaned[columns_to_plot].plot(kind='box', subplots=True, layout=(4, 4), figsize=(15, 15), sharex=False, sharey=False)
plt.tight_layout()
plt.show()

In [ ]:
Q1 = data_cleaned["MEDV"].quantile(0.25)
Q3 = data_cleaned["MEDV"].quantile(0.75)
IQR = Q3 - Q1
seuil_bas = Q1 - 1.5 * IQR
seuil_haut = Q3 + 1.5 * IQR

outliers = data_cleaned[(data_cleaned["MEDV"] < seuil_bas) | (data_cleaned["MEDV"] > seuil_haut)]
print(outliers)

print(f"Nombre d'outliers : {outliers.shape[0]}")

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

sns.histplot(data_cleaned["MEDV"], bins=30, kde=True, ax=axes[0], color="lightblue")
axes[0].set_title("Histogramme de MEDV")

sns.boxplot(y=data_cleaned["MEDV"], ax=axes[1], color="lightblue")
axes[1].set_title("Boxplot de MEDV")

plt.tight_layout()
plt.show()

Beaucoup de valeurs atypiques détectées, pour un grand nombre de variables, notamment pour la target (MEDV : la valeur médiane des maisons occupées par leur propriétaire en milliers de dollars). Nous n'allons pas modifier les variables explicatives à cette étape, nous vérifierons à l'étape de modélisation l'impact de leurs valeurs aberrantes.

In [ ]:
# Passage de Y en log pour réduire l'influence des outliers et faciliter l'interprétation 
import numpy as np

# Transformer MEDV en log
data_cleaned["log_MEDV"] = np.log(data_cleaned["MEDV"])


Q1 = data_cleaned["log_MEDV"].quantile(0.25)
Q3 = data_cleaned["log_MEDV"].quantile(0.75)
IQR = Q3 - Q1
seuil_bas = Q1 - 1.5 * IQR
seuil_haut = Q3 + 1.5 * IQR

outliers = data_cleaned[(data_cleaned["log_MEDV"] < seuil_bas) | (data_cleaned["log_MEDV"] > seuil_haut)]
print(outliers)

print(f"Nombre d'outliers : {outliers.shape[0]}")

import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

sns.histplot(data_cleaned["log_MEDV"], bins=30, kde=True, ax=axes[0], color="lightblue")
axes[0].set_title("Histogramme de log(MEDV)")

sns.boxplot(y=data_cleaned["log_MEDV"], ax=axes[1], color="lightblue")
axes[1].set_title("Boxplot de log(MEDV)")

plt.tight_layout()
plt.show()

In [ ]:
# Suppression de la colonne MEDV
data_cleaned = data_cleaned.drop(columns=['MEDV'])

# Vérification des colonnes restantes
data_cleaned.info()

# Move 'log_MEDV' column to the first position
log_medv = data_cleaned.pop('log_MEDV')
data_cleaned.insert(0, 'log_MEDV', log_medv)

In [ ]:
# Calcul des statistiques descriptives
data_cleaned.drop(columns=['CHAS', 'RAD']).describe()

Interprétations : 

Les statistiques descriptives montrent une grande diversité entre les variables. Par exemple, **log_MEDV** (prix médian des maisons) varie de 1.61 à 3.91, avec une moyenne de 3.03, indiquant une certaine hétérogénéité des prix. En ce qui concerne **CRIM** (taux de criminalité), bien que la moyenne soit relativement modeste à 3.61, l'écart-type élevé de 8.60 reflète des disparités notables entre les zones. **ZN** (proportion de terrain résidentiel) montre également une grande variabilité, allant de 0 à 100 %, tandis que **INDUS** (proportion de terrains industriels) a une moyenne de 11.14 %, mais varie fortement entre 0.46 % et 27.74 %. **NOX** (concentration d'oxydes d'azote) est assez stable avec une moyenne de 0.55 et une faible variabilité. D'autre part, **RM** (nombre de chambres) et **AGE** (âge des maisons) varient respectivement de 3.56 à 8.78 chambres et de 2.9 à 100 ans, montrant une diversité dans les tailles et âges des maisons. **DIS** (distance aux centres d'emploi) varie de 1.13 à 12.13, reflétant des zones proches ou éloignées des centres économiques. Enfin, **TAX** (taux d'imposition) montre une forte disparité avec des valeurs allant de 187 à 711, et **LSTAT** (proportion de population à faible statut socio-économique) varie de 1.73 % à 37.97 %, soulignant des différences sociales importantes. En somme, ces variables révèlent une grande variabilité entre les zones, avec des facteurs comme la criminalité, l'âge des maisons, ou le statut socio-économique.

#### CORRÉLATIONS

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Calcul des corrélations de Spearman entre toutes les variables
correlation_matrix_spearman = data_cleaned.corr(method='spearman')

# Masquer les corrélations dont la valeur absolue est inférieure à 0.6
correlation_matrix_filtered = correlation_matrix_spearman[correlation_matrix_spearman.abs() > 0.6]

# Affichage de la matrice de corrélation filtrée sous forme de heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix_filtered, annot=True, cmap="coolwarm", fmt=".2f", cbar=True, linewidths=0.5)
plt.title("Matrice de Corrélation de Spearman (valeurs > 0.6 en valeur absolue)")
plt.show()

# Afficher la matrice filtrée
correlation_matrix_filtered


Les corrélations de Spearman révèlent plusieurs relations clés entre les variables. Le prix médian des maisons (log_MEDV) est négativement corrélé avec la criminalité (CRIM, -0.53) et fortement lié à la taille des maisons (RM, 0.63), suggérant que des prix plus élevés sont associés à des maisons plus grandes. La criminalité est également fortement liée à la pollution (NOX, 0.82) et à l'indice industriel (INDUS, 0.74), ce qui indique que les zones avec plus de criminalité sont plus polluées et industrialisées. L'âge des maisons (AGE) est lié à une plus grande pollution (NOX, 0.80). De plus, les zones plus éloignées des centres urbains (DIS) ont moins de pollution (NOX, -0.88). La proportion de population à faible statut socio-économique (LSTAT) est fortement négative avec les prix des maisons (log_MEDV, -0.85) et corrélée à un plus petit nombre de chambres (RM, -0.64). Enfin, des taxes foncières élevées (TAX) sont associées à plus de criminalité (CRIM, 0.73) et de pollution (NOX, 0.65).

NOX (concentration des oxydes d'azote) est impliqué dans 3 des 5 relations entre les variables explicatives présentant une corrélation supérieure ou égale à 0,7. Nous allons donc la supprimer.

In [ ]:
# Suppression de la variable NOX
data_cleaned = data_cleaned.drop(columns=['NOX'])

# Vérification des premières lignes après la suppression
print(data_cleaned.head())

### 3)Créer 2 modèles baseline, linéaire et random forest

In [ ]:
# Afficher les informations du DataFrame
print(data_cleaned.info())

# Sélectionner les X quantitatifs
quantitative_columns = data_cleaned.select_dtypes(include=['float64', 'int32','int64']).columns
quantitative_columns = quantitative_columns.difference(['RAD', 'CHAS'])

# Statistiques descriptives pour les variables quantitatives
print("Statistiques descriptives pour les variables quantitatives :")
print(data_cleaned[quantitative_columns].describe())

##### Préparation des données

In [ ]:
# One-hot encoding for 'RAD' column
data_encoded = pd.get_dummies(data_cleaned, columns=['RAD'], prefix='RAD', drop_first=True)
print(data_encoded.head())

In [ ]:
# Rename 'RAD_24' to 'RAD_9'
data_encoded.rename(columns={'RAD_24': 'RAD_9'}, inplace=True)
print(data_encoded.head())

# Convertir les colonnes RAD_2 à RAD_9 de booléen à int (0 ou 1)
cols_to_convert = ['RAD_2', 'RAD_3', 'RAD_4', 'RAD_5', 'RAD_6', 'RAD_7', 'RAD_8', 'RAD_9']

# Convertir en 0 et 1
data_encoded[cols_to_convert] = data_encoded[cols_to_convert].astype(int)

# Vérifier les types après la conversion
print(data_encoded.dtypes)

### 4) Interpréter le modèle linéaire
Utiliser les méthodes intrinsèques du modèle pour l'interprétation

In [ ]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Définition des variables explicatives et de la variable cible
X = data_encoded.drop(columns=['log_MEDV'])  # Variables explicatives
y = data_encoded['log_MEDV']  # Variable dépendante

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajouter une constante pour l'intercept
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

# Estimation du modèle de régression linéaire sur l'ensemble d'entraînement
model = sm.OLS(y_train, X_train).fit()

# Résumé du modèle
print(model.summary())

# Prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Calcul du RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE du modèle : {rmse}")

In [ ]:
# Sélection de variables

import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Séparation des données en ensembles d'entraînement et de test
X = data_encoded.drop(columns=['log_MEDV'])  # Variables explicatives
y = data_encoded['log_MEDV']  # Variable dépendante

# Division des données en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajouter une constante pour l'intercept
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

# Fonction pour effectuer la sélection de variables via backward elimination
def backward_elimination(X_train, y_train, significance_level=0.05):
    initial_features = X_train.columns.tolist()  # Liste des variables initiales
    while True:
        model = sm.OLS(y_train, X_train[initial_features]).fit()  # Estimation du modèle
        p_values = model.pvalues[1:]  # Ignorer la constante
        max_p_value = p_values.max()  # Prendre la plus grande p-value
        if max_p_value > significance_level:  # Si la p-value est supérieure au seuil
            excluded_feature = p_values.idxmax()  # Exclure la variable avec la plus grande p-value
            initial_features.remove(excluded_feature)  # Retirer la variable de la liste
        else:
            break  # Arrêter si toutes les p-values sont inférieures au seuil
    return initial_features

# Appliquer la méthode de sélection de variables
selected_features = backward_elimination(X_train, y_train)

# Modèle final avec les variables sélectionnées
model = sm.OLS(y_train, X_train[selected_features]).fit()

# Résumé du modèle final
print(model.summary())

# Prédictions sur l'ensemble de test
y_pred = model.predict(X_test[selected_features])

# Calcul du RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE du modèle : {rmse}")


Le modèle de régression OLS montre que plusieurs variables explicatives ont un impact significatif sur le prix médian des maisons (log_MEDV). Le R² indique que 77% de la variance de `log_MEDV` est expliquée par les variables indépendantes du modèle. Parmi les variables explicatives, `RM` (nombre moyen de chambres) a l'effet le plus positif et significatif, augmentant le prix médian des maisons de 11.94% pour chaque chambre supplémentaire. D'autres variables, telles que `CRIM` (taux de criminalité) et `LSTAT` (proportion de la population à faible statut socio-économique), ont des effets négatifs significatifs sur les prix, indiquant que des niveaux plus élevés de criminalité ou un statut socio-économique plus faible sont associés à des prix plus bas. La variable `CHAS` (proximité d'une rivière) a également un effet positif, suggérant que les maisons près d'un cours d'eau sont plus chères. De plus, les taxes foncières (`TAX`) et le ratio élèves/professeurs (`PTRATIO`) ont des effets négatifs sur les prix des maisons, tandis que la distance aux centres d'emploi (`DIS`) a également un effet négatif. Enfin, la variable `RAD_3` montre une relation positive avec le prix des maisons. Toutefois, le condition number élevé (7 760) suggère qu'il pourrait y avoir une certaine multicolinéarité entre les variables même après la sélection de variables, ce qui peut affecter la stabilité des estimations des coefficients.

### 5) Tuner votre random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import numpy as np

# Création du pipeline avec standardisation + Random Forest
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardisation des données
    ('rf', RandomForestRegressor(random_state=42))  # Modèle RF sans hyperparamètres fixés
])

# Grille des hyperparamètres à tester
param_grid = {
    'rf__n_estimators': [100, 200, 300],  # Nombre d'arbres
    'rf__max_depth': [None, 10, 20],  # Profondeur maximale
    'rf__min_samples_split': [2, 5, 10],  # Nb min d'échantillons pour une division
    'rf__min_samples_leaf': [1, 2, 4]  # Nb min d'échantillons dans une feuille
}

# Recherche des meilleurs hyperparamètres avec validation croisée (5 folds)
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)  # Entraînement avec les données d'entraînement

# Meilleurs paramètres trouvés
best_params = grid_search.best_params_
print("Meilleurs hyperparamètres :", best_params)

# Meilleur modèle obtenu
best_model = grid_search.best_estimator_

# Prédictions sur l'ensemble de test
y_pred = best_model.predict(X_test)

# Calcul de la RMSE du meilleur modèle sur l'ensemble de test
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE du modèle Random Forest sur l'ensemble de test : {rmse}")

# Calcul de la RMSE du meilleur modèle avec validation croisée (sur les données d'entraînement)
best_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
best_rmse_cv = -np.mean(best_scores)
print(f"RMSE moyen du meilleur modèle Random Forest (Validation croisée) : {best_rmse_cv}")


Le modèle Random Forest présente un bon pouvoir prédictif avec un RMSE de 0.1785 en validation croisée. Les meilleurs hyperparamètres sont : `max_depth` illimité, `min_samples_split` de 10, `min_samples_leaf` de 1, et `n_estimators` à 300. Ce modèle est plus flexible et performant que le modèle linéaire, mais son interprétation nécessite des outils comme la Permutation Feature Importance pour mieux comprendre les interactions et relations non linéaires.

### 6) Interpréter globalement votre meilleur modèle RF 
   1) Utiliser les PDP ou ALE & Permutation feature Importance 
   2) Comparer les résultats du random forest avec votre interprétation du modèle linéaire

In [ ]:
# Permutation Feature Importance
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import permutation_importance
import pandas as pd

# Calcul de l'importance des variables par permutation
perm_importance = permutation_importance(best_model, X_test, y_test, n_repeats=10, random_state=42)

# Visualisation de l'importance des variables
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': perm_importance.importances_mean
})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Affichage des résultats
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.title('Permutation Feature Importance')
plt.xlabel('Importance')
plt.show()

La variable `LSTAT` (le pourcentage de la population de statut socio-économique inférieur) est la plus importante, avec une contribution de plus de 50% à la prédiction de la valeur médiane des maisons. Ensuite, `RM` (le nombre moyen de pièces par logement) joue également un rôle significatif, représentant un peu plus d'1/5 de l'importance. `CRIM` (taux de criminalité) a une importance plus faible, avec une contribution d'un peu moins de 10%. Ensuite l'apport est minime.

In [ ]:
# Affichage des PDP sur la variable avec la plus grande importance (LSTAT)
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import permutation_importance
import pandas as pd

features_to_plot = ['LSTAT']  # Exemple de variables, à adapter

# Créer un seul graphique avec un seul axe
fig, ax = plt.subplots(figsize=(10, 6))

# Affichage des PDP sur l'axe ax
disp = PartialDependenceDisplay.from_estimator(
    best_model, X_train, features=features_to_plot, ax=ax
)

# Ajouter un titre et afficher le graphique
plt.title("Partial Dependence Plots")
plt.show()

La courbe PDP de `LSTAT` montre une relation décroissante entre le statut socio-économique inférieur et la valeur médiane des maisons, ce qui suggère qu'à mesure que la proportion de la population de statut inférieur augmente, la valeur des maisons tend à diminuer. Le palier à 20 indique qu'au-delà de ce seuil, l'effet de `LSTAT` sur la valeur des maisons devient négligeable.

Comparaison des résultats du modèle random forest avec ceux du modèle linéaire : 

Le modèle de régression OLS met en évidence des relations linéaires et simples entre certaines variables et le prix des maisons, comme l'impact positif de `RM` et l'impact négatif de `LSTAT` et `CRIM`. Cependant, bien que ce modèle explique un peu plus des 3/4 de la variance, son interprétation peut être limitée par des problèmes de multicolinéarité, ce qui peut affecter la stabilité des estimations. En comparaison, le modèle Random Forest, avec une erreur de prédiction plus faible, offre une plus grande flexibilité et peut mieux capturer des relations non linéaires et complexes. Par exemple, la variable `LSTAT` est jugée la plus importante dans Random Forest, et la courbe PDP montre qu'au-delà d'un certain seuil de `LSTAT`, son effet devient négligeable, illustrant une dynamique plus subtile qui n'est pas capturée par le modèle linéaire.

### 7) Réaliser une explicabilité par individu sur le modèle RF
- 1) ICE, le PDP est-il une bonne représentation des variables importantes de votre modèle?
- 2) LIME
- 3) SHAP watterfall plot

In [ ]:
# ICE
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay
import numpy as np

# Sélection de la variable 'LSTAT' (par exemple)
feature = 'LSTAT'

# Créer un cadre (figure) pour afficher le graphique sans doublon
fig, ax = plt.subplots(figsize=(10, 6))

# Utiliser 'from_estimator' pour générer les plots sans créer un cadre supplémentaire
disp = PartialDependenceDisplay.from_estimator(best_model, X_train, features=[X_train.columns.get_loc(feature)], kind="individual", ax=ax)

# Titre du graphique
plt.title(f"ICE plots pour la variable {feature}")
plt.show()


Dans la plupart des cas, les courbes ICE ont une trajectoire similaire, ce qui indique que l'effet observé est relativement homogène parmi les observations (augmentation de la proportion de la population de statut socio-économique inférieur tend à diminuer la valeur des maisons). Dans le cas actuel le PDP est une bonne représentation, même si les courbes ICE améliorent la compréhension des effets individuels.

In [ ]:
#LIME
import lime
from lime.lime_tabular import LimeTabularExplainer

# Initialisation de LIME pour un modèle tabulaire
explainer = LimeTabularExplainer(X_train.values, feature_names=X_train.columns, class_names=['target'], discretize_continuous=True)

# Choix d'un individu à expliquer (par exemple le premier de l'ensemble de test)
idx = 0  # Index de l'individu à expliquer
exp = explainer.explain_instance(X_test.iloc[idx].values, best_model.predict, num_features=5)

# Visualisation de l'explication
exp.show_in_notebook()

Erreur qui indique que LIME ne prend pas en charge les modèles de régression (comme RandomForestRegressor) dans sa version actuelle.

In [ ]:
#SHAP
import shap

# Explainer SHAP basé sur l'arbre RandomForestRegressor ajusté
explainer = shap.TreeExplainer(best_model.named_steps['rf'])

# Choisir un échantillon de test à expliquer
shap_values = explainer.shap_values(X_test)

# Afficher le SHAP waterfall plot pour un échantillon spécifique
shap.initjs()  # Initialiser les visualisations JavaScript
shap.waterfall_plot(shap_values[0])  # Utiliser shap_values[0] pour un exemple

J'ai essayé plusieurs fois de corriger l'erreur mais sans parvenir à obtenir le graphique attendu, je suis désolé. 

### 8) Explorer les graphiques SHAP étudiés  dans la partie CM
   1) beeswarm (Contribution des variables)
   2) scatter (équivalent pdp) - déja représenté dans la section "6) Interpréter globalement votre modèle meilleur modèle RF "

In [ ]:
# Beeswarm Plot 
import shap

# Calcul des valeurs SHAP pour le modèle
explainer = shap.TreeExplainer(best_model.named_steps['rf'])  # Utilisation du modèle Random Forest du pipeline
shap_values = explainer.shap_values(X_train)  # Valeurs SHAP pour les données d'entraînement

# Visualisation avec un beeswarm plot
shap.summary_plot(shap_values, X_train)

# Calculer la moyenne des valeurs SHAP pour chaque variable
mean_shap_values = shap_values.mean(axis=0)

# Créer un DataFrame pour stocker les résultats
shap_summary = pd.DataFrame({
    'Feature': X_train.columns,  # Nom de la caractéristique
    'Mean SHAP': mean_shap_values  # Moyenne des valeurs SHAP (avec signe)
})

# Trier par ordre décroissant de l'importance SHAP (en valeur absolue)
shap_summary = shap_summary.sort_values(by='Mean SHAP', ascending=False)
print(shap_summary)

Les variables les plus influentes sur le prix des maisons sont principalement négatives, avec un impact particulièrement fort de **LSTAT**, qui montre qu'une proportion plus élevée de la population à faible statut socio-économique diminue significativement la valeur des propriétés. Des facteurs tels que **CRIM** (criminalité) et **PTRATIO** (ratio élèves/enseignants) ont eux aussi un effet négatif sur les prix des maisons. La variables **RM** (nombre moyen de chambres) est la plus pertinente pour expliquer une variation positive du prix des maisons. Cependant, la superposition fréquente des couleurs sur le Beeswarm Plot indique que pour plusieurs facteurs la relation avec le prix des maisons n'est pas monotone. Cela suggère des relations complexes et non linéaires, mieux capturées par des modèles comme Random Forest.